# Import Librarries

In [ ]:
from pypdf import PdfReader
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

# Parse Document

In [ ]:
documents_1 = ''

reader = PdfReader('/workspaces/digital_research_guide/1706.03762v7.pdf')
for page in reader.pages:
    documents_1 += page.extract_text()

In [ ]:
cleaned_string = documents_1.replace('\n', ' ')

In [ ]:
first_section = "Abstract"
ignore_after = "References"

In [ ]:
cleaned_string

In [ ]:
start_index = cleaned_string.find(first_section)
end_index = cleaned_string.rindex(ignore_after)

In [ ]:
start_index,end_index 

In [ ]:
cleaned_string = cleaned_string[start_index:end_index]

In [ ]:
sentence_list = cleaned_string.split('. ')

In [ ]:
context_list = []

In [ ]:
group_size = 20
overlap = 5
i = 0 
while True:
    end_index = min(len(sentence_list),(i+group_size))
    group = sentence_list[i:i+group_size]
    text = '. '.join(group)
    context_list.append(text)
    i+=group_size-overlap
    if i>=len(sentence_list):
        break


# Create Embedding

In [ ]:
check_point = 'mixedbread-ai/mxbai-embed-large-v1'

In [ ]:
model = SentenceTransformer(check_point)

In [ ]:
max_seq_length = model.get_max_seq_length()
embedding_dimension = model.get_sentence_embedding_dimension()

In [ ]:
def get_embeddings(doc):
    model_input = doc
    out =  model.encode(model_input)
    return out

In [ ]:
embeddings = list(map(get_embeddings,context_list))

In [ ]:
embeddings_array = np.array(embeddings)

# Create FAISS Vector Store

In [ ]:
index = faiss.IndexFlatL2(model.get_sentence_embedding_dimension())
index.add(embeddings_array)

In [ ]:
faiss.write_index(index, 'doc.index')